In [45]:
# Setup project name and directory.
## Project name is a global variable, and will prefix all output files.
## Project name directory is created within current directory.
def setup_project(name):
    import os
    os.mkdir(os.path.expanduser(name))
    global project_name
    project_name = name

# CSV to Dict
## In: CSV file, with any number of columns, first row must have column names.
## Out: Two dicts, one keyed Attributes, has a list of column names; the other keyed Content has integer keys, each entry is itself a dict of row values, keyed by column names.
def CSV_to_dict(csv_file, delimiter_character):
    raw_data = {}
    import csv
    with open(csv_file) as file:
        data = csv.reader(file, delimiter = delimiter_character)
        j = 0
        for i in data: 
            if j == 0: 
                attribute_names = i
            if j > 0:
                raw_data[j-1] = dict()
                for k in range(0,len(attribute_names)):
                    raw_data[j-1][attribute_names[k]] = i[k]
            j = j + 1
    return { 'Attributes': attribute_names, 'Content': raw_data }

# Shorten input string to n characters and ellipsis '...'.
def shorten_string(string, n):
    if len(string) < n: return string
    else: return string[0:n] + '...'

# Make and describe nodes. Description preserves input data content. Return dict.
## node_id is a unique node identifier, uniqueness based on dict key, with key an integer.
## full_name is the text associated toon the node, read from input data.
## nickname are the first 15 or less characters of full_name. Easier to draw in figures.
## instance_of is the name of the column
def make_nodes_dict(def_data):
    node_id = 1
    nodes_dict = dict()
    for a in def_data['Attributes']:
        for c in def_data['Content']:
            full_name = def_data['Content'][c][a]
            nodes_dict[node_id] = { 'node_id': project_name + '_n_' + str(node_id), 
                                   'full_name': full_name,
                                   'nickname': shorten_string(full_name, 15),
                                   'instance_of': a,
                                  }
            node_id = node_id + 1
    return nodes_dict



In [11]:
setup_project('SRN')

In [57]:
y = make_nodes_dict(CSV_to_dict('raw_data/ilang_SRN_data.csv', ','))

In [18]:
# Define classes of relationships over column names (Attributes) in imported data.
relationship_classes = dict()
## A definiens defines a definiendum.
relationship_classes[(defs['Attributes'][2], defs['Attributes'][0])] = 'Defines'
## A short definiendum abbreviates a definiendum.
relationship_classes[(defs['Attributes'][1], defs['Attributes'][0])] = 'Abbreviates'
relationship_classes

{('Definiens', 'Definiendum'): 'Defines',
 ('Short Definiendum', 'Definiendum'): 'Abbreviates'}

In [59]:
d = CSV_to_dict('raw_data/ilang_SRN_data.csv', ',')

In [60]:
d

{'Attributes': ['Definiendum', 'Short Definiendum', 'Definiens'],
 'Content': {0: {'Definiendum': 'SRN',
   'Short Definiendum': 'SRN',
   'Definiens': 'Smart Retail Network software.'},
  1: {'Definiendum': 'SDM',
   'Short Definiendum': 'SDM',
   'Definiens': 'Shop Display Management software.'},
  2: {'Definiendum': 'Shop Display',
   'Short Definiendum': 'Shop Display',
   'Definiens': 'Object installed at a POP, which is used to promote products; Shop Display is synonym with Fixture.'},
  3: {'Definiendum': 'Basic Shop Display',
   'Short Definiendum': 'BSD',
   'Definiens': 'A Shop Display which cannot itself be subdivided.'},
  4: {'Definiendum': 'Complex Shop Display',
   'Short Definiendum': 'CSD',
   'Definiens': 'A Shop Display made from two or more Basic Shop Displays.'},
  5: {'Definiendum': 'Shop Display Identifier',
   'Short Definiendum': 'SDID',
   'Definiens': 'Unique numeric identifier generated by SDM for each BSD, used to identify a BSD when a QR code scanner is no

In [71]:
i = 1
o = dict()
out = dict()
for c in d['Content']:
    for a in d['Attributes']:
        full_name = d['Content'][c][a]
        for orig, dest in relationship_classes:
            if orig == d['Content'][c][a] and 
        out[i] = { 'node_id': project_name + '_n_' + str(i), 
                  'full_name': full_name,
                  'nickname': shorten_string(full_name, 15),
                  'instance_of': a,
                 }
        print(full_name, c, a)


# for a in def_data['Attributes']:
#     for c in def_data['Content']:
#         full_name = def_data['Content'][c][a]
#         nodes_dict[node_id] = { 'node_id': project_name + '_n_' + str(node_id), 
#                                'full_name': full_name,
#                                'nickname': shorten_string(full_name, 15),
#                                'instance_of': a,
#                               }
#         node_id = node_id + 1

SRN 0 Definiendum
SRN 0 Short Definiendum
Smart Retail Network software. 0 Definiens
SDM 1 Definiendum
SDM 1 Short Definiendum
Shop Display Management software. 1 Definiens
Shop Display 2 Definiendum
Shop Display 2 Short Definiendum
Object installed at a POP, which is used to promote products; Shop Display is synonym with Fixture. 2 Definiens
Basic Shop Display 3 Definiendum
BSD 3 Short Definiendum
A Shop Display which cannot itself be subdivided. 3 Definiens
Complex Shop Display 4 Definiendum
CSD 4 Short Definiendum
A Shop Display made from two or more Basic Shop Displays. 4 Definiens
Shop Display Identifier 5 Definiendum
SDID 5 Short Definiendum
Unique numeric identifier generated by SDM for each BSD, used to identify a BSD when a QR code scanner is not available. 5 Definiens
Shop Display Supplier 6 Definiendum
Shop Display Supplier 6 Short Definiendum
Representative of the company supplying Shop Displays. 6 Definiens
Point of Purchase 7 Definiendum
POP 7 Short Definiendum
Location (

'Definiendum'

In [73]:
d['Attributes'][0]

'Definiendum'

In [76]:
d['Content'][5]

{'Definiendum': 'Shop Display Identifier',
 'Short Definiendum': 'SDID',
 'Definiens': 'Unique numeric identifier generated by SDM for each BSD, used to identify a BSD when a QR code scanner is not available.'}

In [90]:
for e in d['Content'][5]:
    print(e)

Definiendum
Short Definiendum
Definiens


In [87]:
for a1 in d['Attributes']:
    for a2 in d['Attributes']:
        print('rel_names['+'\'' + a1 + '\'' + ',' + '\'' + a2 + '\'' + '] = ')

rel_names['Definiendum','Definiendum'] = 
rel_names['Definiendum','Short Definiendum'] = 
rel_names['Definiendum','Definiens'] = 
rel_names['Short Definiendum','Definiendum'] = 
rel_names['Short Definiendum','Short Definiendum'] = 
rel_names['Short Definiendum','Definiens'] = 
rel_names['Definiens','Definiendum'] = 
rel_names['Definiens','Short Definiendum'] = 
rel_names['Definiens','Definiens'] = 


In [88]:
rel_names = dict()
rel_names['Definiendum','Definiendum'] = None
rel_names['Definiendum','Short Definiendum'] = 'Abbreviated by'
rel_names['Definiendum','Definiens'] = 'Defined by'
rel_names['Short Definiendum','Definiendum'] = 'Abbreviates'
rel_names['Short Definiendum','Short Definiendum'] = None
rel_names['Short Definiendum','Definiens'] = None
rel_names['Definiens','Definiendum'] = 'Defines'
rel_names['Definiens','Short Definiendum'] = None
rel_names['Definiens','Definiens'] = None

In [89]:
rel_names

{('Definiendum', 'Definiendum'): None,
 ('Definiendum', 'Short Definiendum'): 'Abbreviated by',
 ('Definiendum', 'Definiens'): 'Defined by',
 ('Short Definiendum', 'Definiendum'): 'Abbreviates',
 ('Short Definiendum', 'Short Definiendum'): None,
 ('Short Definiendum', 'Definiens'): None,
 ('Definiens', 'Definiendum'): 'Defines',
 ('Definiens', 'Short Definiendum'): None,
 ('Definiens', 'Definiens'): None}

In [124]:
rels = dict()
for e in d['Content'][5]:
    for f in d['Content'][5]:
        if rel_names[e,f] != None:
            print(e,f)
            rels[d['Content'][5][e], rel_names[e,f]] = d['Content'][5][f]

Definiendum
Definiendum Short Definiendum
Definiendum Definiens
Short Definiendum
Short Definiendum Definiendum
Definiens
Definiens Definiendum


In [123]:
rels

{('Shop Display Identifier', 'Abbreviated by'): 'SDID',
 ('Shop Display Identifier',
  'Defined by'): 'Unique numeric identifier generated by SDM for each BSD, used to identify a BSD when a QR code scanner is not available.',
 ('SDID', 'Abbreviates'): 'Shop Display Identifier',
 ('Unique numeric identifier generated by SDM for each BSD, used to identify a BSD when a QR code scanner is not available.',
  'Defines'): 'Shop Display Identifier'}

In [138]:
def make_nodes_dict_2(def_data):
    node_id = 1
    nodes_dict = dict()
    for i in def_data['Content']:
        rels = dict()
        for c1 in def_data['Content'][i]:
            for c2 in def_data['Content'][i]:
                print(c1,c2)
                if rel_names[c1,c2] != None:
                    rels[d['Content'][i][c1], rel_names[c1,c2]] = def_data['Content'][i][c2]
        for a in def_data['Attributes']:
            full_name = def_data['Content'][i][a]
            nodes_dict[node_id] = { 'node_id': project_name + '_n_' + str(node_id), 
                                   'full_name': full_name,
                                   'nickname': shorten_string(full_name, 15),
                                   'instance_of': a,
                                   'relationships': rels
                                  }
            node_id = node_id + 1
        del rels
    return nodes_dict

In [139]:
q = make_nodes_dict_2(d)

Definiendum Definiendum
Definiendum Short Definiendum
Definiendum Definiens
Short Definiendum Definiendum
Short Definiendum Short Definiendum
Short Definiendum Definiens
Definiens Definiendum
Definiens Short Definiendum
Definiens Definiens
Definiendum Definiendum
Definiendum Short Definiendum
Definiendum Definiens
Short Definiendum Definiendum
Short Definiendum Short Definiendum
Short Definiendum Definiens
Definiens Definiendum
Definiens Short Definiendum
Definiens Definiens
Definiendum Definiendum
Definiendum Short Definiendum
Definiendum Definiens
Short Definiendum Definiendum
Short Definiendum Short Definiendum
Short Definiendum Definiens
Definiens Definiendum
Definiens Short Definiendum
Definiens Definiens
Definiendum Definiendum
Definiendum Short Definiendum
Definiendum Definiens
Short Definiendum Definiendum
Short Definiendum Short Definiendum
Short Definiendum Definiens
Definiens Definiendum
Definiens Short Definiendum
Definiens Definiens
Definiendum Definiendum
Definiendum Shor

In [140]:
q

{1: {'node_id': 'SRN_n_1',
  'full_name': 'SRN',
  'nickname': 'SRN',
  'instance_of': 'Definiendum',
  'relationships': {('SRN', 'Abbreviated by'): 'SRN',
   ('SRN', 'Defined by'): 'Smart Retail Network software.',
   ('SRN', 'Abbreviates'): 'SRN',
   ('Smart Retail Network software.', 'Defines'): 'SRN'}},
 2: {'node_id': 'SRN_n_2',
  'full_name': 'SRN',
  'nickname': 'SRN',
  'instance_of': 'Short Definiendum',
  'relationships': {('SRN', 'Abbreviated by'): 'SRN',
   ('SRN', 'Defined by'): 'Smart Retail Network software.',
   ('SRN', 'Abbreviates'): 'SRN',
   ('Smart Retail Network software.', 'Defines'): 'SRN'}},
 3: {'node_id': 'SRN_n_3',
  'full_name': 'Smart Retail Network software.',
  'nickname': 'Smart Retail Ne...',
  'instance_of': 'Definiens',
  'relationships': {('SRN', 'Abbreviated by'): 'SRN',
   ('SRN', 'Defined by'): 'Smart Retail Network software.',
   ('SRN', 'Abbreviates'): 'SRN',
   ('Smart Retail Network software.', 'Defines'): 'SRN'}},
 4: {'node_id': 'SRN_n_4',
